In [31]:

from pydantic import BaseModel
from typing import Optional
from groq import Groq
import json
import instructor
from typing import List, Optional, Literal


# Initialize the client with instructor
api_key = "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"
client = Groq(api_key=api_key)


### INSTRUCTOR BASED APPROACH

In [32]:
client = instructor.from_groq(client)

class User(BaseModel):
    name: str
    age: int
    email: Optional[str] = None

In [33]:
class SupportTicket(BaseModel):
    ticket_id: str
    order_id: str
    user_id: str
    date: str
    description: str
    category: Literal["delivery", "payment", "other"]
    priority: Optional[Literal["low", "medium", "high"]] = None
    status: Optional[Literal["open", "closed", "in progress"]] = None
    
    

In [34]:

# Now you can use the Pydantic model directly with response_model
completion = client.chat.completions.create(
    model="gemma2-9b-it",
    messages=[
        {
            "role": "system",
            "content": """You are an AI assistant that helps users with their tasks."""
        },
        {
            "role": "user",
            "content": """Make up a user."""
        }
    ],
    temperature=0.6,
    max_completion_tokens=1024,
    response_model=User  # Use response_model instead of response_format
)

print(completion)  # This will be a User instance

name='Alice' age=30 email=None


In [35]:

# Now you can use the Pydantic model directly with response_model
completion = client.chat.completions.create(
    model="gemma2-9b-it",
    messages=[
        {
            "role": "system",
            "content": """You are a support ticket assistant working in a food delivery company. You will be given a description /
             of a support ticket raised by a user. Your task is to extract the following information from the description: /
             
             1. ticket_id   
             2. order_id
             3. user_id
             4. date
             5. description (create a summary of the user's ticket)
             6. category (e.g. delivery, payment, etc.)
             7. priority (e.g. low, medium, high)
             8. status (e.g. open, closed, in progress)

             DO NOT include any additional information.
             
             """
        },
        {
            "role": "user",
            "content": """ I ordered a chicken biriyani and beef. but beef is missing. order was placed on 2023-01-01. ticket_id is 1232399, order_id is 12312399, user_id is 12322399"""
        }
    ],
    temperature=0.6,
    max_completion_tokens=1024,
    response_model=SupportTicket  # Use response_model instead of response_format
)

print(completion)  # This will be a User instance

ticket_id='1232399' order_id='12312399' user_id='12322399' date='2023-01-01' description='User ordered chicken biriyani and beef, but beef was missing from the order.' category='delivery' priority='high' status='open'
